                                  Mini Project 11-Zomato Recommender System

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")
import re
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv("zomato.csv")
df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,https://www.zomato.com/bangalore/best-brews-fo...,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,080 40301477,Whitefield,Bar,NaN,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,https://www.zomato.com/bangalore/vinod-bar-and...,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,+91 8197675843,Whitefield,Bar,NaN,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,https://www.zomato.com/bangalore/plunge-sherat...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,NaN,Whitefield,Bar,NaN,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,https://www.zomato.com/bangalore/chime-sherato...,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,080 49652769,"ITPL Main Road, Whitefield",Bar,"Cocktails, Pizza, Buttermilk",Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [4]:
df.isnull().sum()

url                                0
address                            0
name                               0
online_order                       0
book_table                         0
rate                            7775
votes                              0
phone                           1208
location                          21
rest_type                        227
dish_liked                     28078
cuisines                          45
approx_cost(for two people)      346
reviews_list                       0
menu_item                          0
listed_in(type)                    0
listed_in(city)                    0
dtype: int64

# EDA, Data Cleaning and Feature Engineering

Now the next step is data cleaning and feature engineering for this step we need to do a lot of stuff with the data such as:


Deleting Unnecessary Columns

Removing the Duplicates

Remove the NaN values from the dataset

Changing the column names

Data Transformations

Data Cleaning

Adjust the column names Now, let’s perform all the above steps in our data:

In [5]:
#Dropping the column "dish_liked", "phone", "url" and saving the new dataset as "zomato"
zomato=df.drop(['url','phone','dish_liked'],axis=1) 
print(zomato.head())


                                             address                   name  \
0  942, 21st Main Road, 2nd Stage, Banashankari, ...                  Jalsa   
1  2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...         Spice Elephant   
2  1112, Next to KIMS Medical College, 17th Cross...        San Churro Cafe   
3  1st Floor, Annakuteera, 3rd Stage, Banashankar...  Addhuri Udupi Bhojana   
4  10, 3rd Floor, Lakshmi Associates, Gandhi Baza...          Grand Village   

  online_order book_table   rate  votes      location            rest_type  \
0          Yes        Yes  4.1/5    775  Banashankari        Casual Dining   
1          Yes         No  4.1/5    787  Banashankari        Casual Dining   
2          Yes         No  3.8/5    918  Banashankari  Cafe, Casual Dining   
3           No         No  3.7/5     88  Banashankari          Quick Bites   
4           No         No  3.8/5    166  Basavanagudi        Casual Dining   

                         cuisines approx_cost(for two pe

In [6]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [7]:
print("Shape before removing duplicates:", zomato.shape)
zomato.drop_duplicates(inplace=True)
print("Shape after removing duplicates:", zomato.shape)

Shape before removing duplicates: (51674, 14)
Shape after removing duplicates: (51674, 14)


In [8]:
#Removing the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)
print("Shape before removing NaN values:", zomato.shape)
zomato.dropna(how='any', inplace=True)
print("Shape after removing NaN values:", zomato.shape)


Shape before removing NaN values: (43499, 14)
Shape after removing NaN values: (43499, 14)


In [9]:
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})
print(zomato.columns)


Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')


In [10]:
#Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['cost'] = zomato['cost'].astype(float)
print(zomato['cost'].dtype)


float64


In [11]:
# Removing '/5' from Rates
zomato = zomato.loc[zomato.rate != 'NEW']
zomato = zomato.loc[zomato.rate != '-'].reset_index(drop=True)

remove_slash = lambda x: x.replace('/5', '') if type(x) == str else x
zomato['rate'] = zomato['rate'].apply(remove_slash).str.strip().astype(float)
print(zomato['rate'].dtype)
print(zomato['rate'].head())


float64
0    4.1
1    4.1
2    3.8
3    3.7
4    3.8
Name: rate, dtype: float64


In [12]:
# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)
print(zomato['name'].head())
print(zomato['online_order'].head())
print(zomato['book_table'].head())


0                    Jalsa
1           Spice Elephant
2          San Churro Cafe
3    Addhuri Udupi Bhojana
4            Grand Village
Name: name, dtype: object
0     True
1     True
2     True
3    False
4    False
Name: online_order, dtype: bool
0     True
1    False
2    False
3    False
4    False
Name: book_table, dtype: bool


In [13]:
## Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)
print(zomato['Mean Rating'].head())


0    3.99
1    3.97
2    3.58
3    3.45
4    3.58
Name: Mean Rating, dtype: float64


Now in the next step, we perform  text preprocessing steps:

Lower casing

Removal of Punctuations

Removal of Stopwords

Removal of URLs

Spelling correction



In [14]:
## Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()
print(zomato["reviews_list"].head())


0    [('rated 4.0', 'rated\n  a beautiful place to ...
1    [('rated 4.0', 'rated\n  had been here for din...
2    [('rated 3.0', "rated\n  ambience is not that ...
3    [('rated 4.0', "rated\n  great food and proper...
4    [('rated 4.0', 'rated\n  very good restaurant ...
Name: reviews_list, dtype: object


In [15]:
# Defining a function to remove punctuation using regular expressions
def remove_punctuation(text):
    """Remove punctuation from the text"""
    return re.sub(r'[^\w\s]', '', text)

# Applying the function to the "reviews_list" column
zomato["reviews_list"] = zomato["reviews_list"].apply(remove_punctuation)
print(zomato["reviews_list"].head())

0    rated 40 ratedn  a beautiful place to dine int...
1    rated 40 ratedn  had been here for dinner with...
2    rated 30 ratedn  ambience is not that good eno...
3    rated 40 ratedn  great food and proper karnata...
4    rated 40 ratedn  very good restaurant in neigh...
Name: reviews_list, dtype: object


In [16]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

In [17]:
## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)


,reviews_list,cuisines
11877,rated 50 ratedn brilliant medoken every bite m...,"Desserts, Beverages"
17280,rated 30 ratedn good go place healthy breakfas...,Cafe
29654,rated 50 ratedn id come across cafe app called...,"Fast Food, Beverages"
3286,rated 40 ratedn name goesit real zaikhathe foo...,"Chinese, North Indian, Mughlai"
29022,rated 50 ratedn hardcore south indian food lov...,"South Indian, North Indian, Chinese, Street Fo..."


In [18]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]
    
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)


In [19]:
# Randomly sample 60% of your dataframe
df_percent = zomato.sample(frac=0.5)

In [20]:
zomato.head()

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,Banashankari,"North Indian, Mughlai, Chinese",800.0,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99
1,Spice Elephant,True,False,4.1,Banashankari,"Chinese, North Indian, Thai",800.0,rated 40 ratedn dinner family turned good choo...,Banashankari,3.97
2,San Churro Cafe,True,False,3.8,Banashankari,"Cafe, Mexican, Italian",800.0,rated 30 ratedn ambience good enough pocket fr...,Banashankari,3.58
3,Addhuri Udupi Bhojana,False,False,3.7,Banashankari,"South Indian, North Indian",300.0,rated 40 ratedn great food proper karnataka st...,Banashankari,3.45
4,Grand Village,False,False,3.8,Basavanagudi,"North Indian, Rajasthani",600.0,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,3.58


In [21]:
zomato.sample(5)

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
20049,Indulge By Innerchef,True,False,3.1,Hennur,"Desserts, Bakery",400.0,rated 40 ratedn ordered oreo mudpie tart fudge...,Kammanahalli,3.18
11365,11 To 11 Express Biriyanis,True,False,3.5,Electronic City,"Biryani, Kebab",300.0,rated 10 ratedn food quantity satisfied per co...,Electronic City,3.19
21662,Juiced Burger,True,False,3.5,South Bangalore,Burger,750.0,rated 50 ratedn burger yummy juicy fresh taste...,Koramangala 4th Block,3.19
32055,The Kabab Studio - Goldfinch Hotel,False,True,4.2,Seshadripuram,"North Indian, Chinese, Continental",1.5,rated 50 ratedn worth visiting family get toge...,Malleshwaram,4.10
29843,Jimi'S Beer Cafe,False,False,3.8,Koramangala 6th Block,"American, North Indian, Chinese",1.8,rated 50 ratedn great place great beverages gr...,Koramangala 7th Block,3.63


In [22]:
zomato.shape

(41237, 10)

In [23]:
zomato.columns

Index(['name', 'online_order', 'book_table', 'rate', 'location', 'cuisines',
       'cost', 'reviews_list', 'city', 'Mean Rating'],
      dtype='object')

In [24]:
df_percent.shape

(20618, 10)

# TF-IDF Vectorization


In [25]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)


In [26]:
# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

Now the last step for creating a Restaurant Recommendation System is to write a function that will recommend restaurants:


In [27]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df_percent.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df_percent[['cuisines','Mean Rating', 'cost']][df_percent.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new
recommend('Pai Vihar')

TOP 7 RESTAURANTS LIKE Pai Vihar WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Samosa Singh,"North Indian, Mithai",3.60,250.0
Samosa Singh,"Street Food, Beverages",3.60,150.0
Prasiddhi Food Corner,"Fast Food, North Indian, South Indian",3.45,200.0
Shrusti Coffee,"Cafe, South Indian",3.45,150.0
Shanthi Sagar,"South Indian, North Indian, Chinese",3.44,400.0
Mayura Sagar,"Chinese, North Indian, South Indian",3.32,250.0
Vasanth Vihar - Since 1965,"South Indian, Street Food",3.32,150.0


In this mini project, we developed a content-based restaurant recommender system using Zomato data. We began by preprocessing the data, including handling missing values, text normalization, and feature engineering. Employing TF-IDF and cosine similarity, we successfully recommended similar restaurants based on user-entered preferences. While the system provides promising recommendations, ongoing refinement and evaluation are crucial for ensuring fairness, relevance, and user satisfaction. Further enhancements could involve incorporating user feedback mechanisms and exploring collaborative filtering techniques for more personalized recommendations.